In [9]:
pip install okx

Note: you may need to restart the kernel to use updated packages.


# 要購買的價格

In [20]:
from okx.api import Account
from okx.api import Trade
from okx_api import Market as Market_api
from pprint import pprint
import json
import os
import time
import math
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
KEY = os.getenv('key')
SECRET = os.getenv('secret')
PASSPHRASE = os.getenv('passphrase')

In [21]:
market = Market_api(key = '', secret = '', passphrase = '', flag = '0')
account = Account(
    # 交易账户需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)
# 实例化trade
trade = Trade(
    # 交易模块需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)

## 邏輯
1. 獲取當下價格計算和前幾分鐘前的價格差
2. 若價格漲跌超過特定趴數，被視為不合理
3. 使用25%資金開倉
4. 追蹤開倉狀態，若沒有成功則取消訂單，反之設定止盈0.1%止損0.1%
5. 設定標記，不會再有持倉的情況下在每個迴圈加倉
6. 資金永遠大於30u
7. 槓桿隨著震幅增加

### 增加
1. 當該幅度持續上漲或下跌，則使用25%資金但是更高的槓桿開單
2. 注意第二次的資金應該為33.3%才會是第一次的資金


### 問題
1. 開新訂單的同時修改原有訂單的止盈止損是否會有更好的效果

In [13]:
def Set_Leverage(INSTID, ISOLATED, LEVER):
    
    set_long = account.set_leverage(
        instId = INSTID, 
        mgnMode = ISOLATED, 
        lever = LEVER, 
        posSide = 'long'
    )
    set_short =account.set_leverage(
        instId = INSTID, 
        mgnMode = ISOLATED, 
        lever = LEVER, 
        posSide = 'short'
    )
    
    return set_long, set_short

In [19]:
while True:
    # 標記一次開倉
    long_mark = True
    short_mark = True
    
    result = market.get_history_index_candles(instId = "INJ-USDT", bar = "1m", limit = "10")
    result['data']
    #最新幣價對十根k線之前的幣價進行計算
    val = (float(result['data'][0][1]) - float(result['data'][9][1]))/float(result['data'][9][1])*100
    price_now = result['data'][0][1]
    print(f"當下價格: {price_now}")
    print(f"震幅: {round(val, 3)}%")

    balance_result = account.get_balance(ccy = 'USDT')
    bal = balance_result['data'][0]['details'][0]['availBal']
    print(f"USDT balance: {bal}")

    # 設定每次交易只拿資產的 25% 操作
    # 總共可買入的幣，乘以 25%
    # 總共買入 px = total 個，價格為 result['data'][0][1]
    total = float(bal)/float(result['data'][0][1])*0.25
        
    # 確保資金永遠高於30u
    if float(bal) > 30:

        if val > 1.5:
            LEVER = 15

        elif val > 1:
            LEVER = 10

        elif val > 0.5:
            LEVER = 5

        else:
            LEVER = 1

        # 槓桿
        set_leverage_result = Set_Leverage('INJ-USDT-SWAP', 'isolated', LEVER)

        # 判斷能否開倉的規則是沒有持倉
        positions_result = account.get_positions(instType='SWAP')
        if len(positions_result['data']) > 1: # 持倉為兩個是極限，但是開空兩次歲視同一個倉位，所以追蹤倉位的保證金

            short_mark = False
            long_mark = False

            print("持倉中，不可開倉") 

        else: # 沒有持倉、一個持倉

            short_mark = True
            long_mark = True

            positions_result = account.get_positions(instType='SWAP')
            
            # 保證金和持有資金的比例為1:1(因為開倉兩次，各開25%)
            if round(float(positions_result['data'][0]['margin'])/float(bal), 2) == 0.67: # 扣掉是為了誤差
                
                print("沒有持倉或有一個持倉，可開倉")

            elif round(float(positions_result['data'][0]['margin'])/float(bal), 2) == 0:
                
                print("沒有持倉，可開倉")


        if val > 0.5 and short_mark:

            # 開空
            set_order_swap_limit_open_result = trade.set_order(
                instId = 'INJ-USDT-SWAP',
                ordType = 'limit',
                tdMode = 'isolated', # isolated:逐倉 cross:全倉
                posSide = 'short', # 空單對應到 sell
                side = 'sell',
                sz = str(math.floor(total*10)),
                px = str(float(result['data'][0][1])),
                tpTriggerPx = str(float(result['data'][0][1])*0.99), # 止盈
                tpOrdPx = '-1',
                slTriggerPx = str(float(result['data'][0][1])*1.01), # 止損
                slOrdPx = '-1'
            )

            # 等待10秒，看有沒有持倉
            time.sleep(10)

            # 追蹤訂單
            order_result = trade.get_order(
                instId = 'INJ-USDT-SWAP',
                ordId = set_order_swap_limit_open_result['data'][0]['ordId']
            )
            print('data', order_result['data'])

            # 若沒有成功持倉，撤單，等待下一個迴圈重新判斷
            if order_result['data'][0]['state'] == "live":

                # 按照instId与ordId取消订单
                set_cancel_order_result = trade.set_cancel_order(
                    instId = 'INJ-USDT-SWAP', ordId = set_order_swap_limit_open_result['data'][0]['ordId']
                )
                print("沒有成功持空倉，撤單")

            # 若成功持倉，設置平倉價格為當下的價格下跌0.1%，以及止損價格0.1%
            elif order_result['data'][0]['state'] == "filled":
                # 不會重複開倉
                long_mark = False
                
                print(f"已經設定空單止盈止損，槓桿為 {LEVER}")

            # 緩衝
            time.sleep(50)

        elif val < -0.5 and long_mark:
            
            # 開多
            set_order_swap_limit_open_result = trade.set_order(
                instId = 'INJ-USDT-SWAP',
                ordType = 'limit',
                tdMode = 'isolated', # isolated:逐倉 cross:全倉
                posSide = 'long', # 多單對應到 buy
                side = 'buy',
                sz = str(math.floor(total*10)),
                px = str(float(result['data'][0][1])),
                tpTriggerPx = str(float(result['data'][0][1])*1.01), # 止盈
                tpOrdPx = '-1',
                slTriggerPx = str(float(result['data'][0][1])*0.99), # 止損
                slOrdPx = '-1'
            )

            # 等待10秒，看有沒有持倉
            time.sleep(10)

            # 追蹤訂單，重新呼叫get_order
            order_result = trade.get_order(
                instId = 'INJ-USDT-SWAP',
                ordId = set_order_swap_limit_open_result['data'][0]['ordId']
            )
            print('data', order_result['data'])
            
            # 若沒有成功持倉，撤單，等待下一個迴圈重新判斷
            if order_result['data'][0]['state'] == "live":  # 需更改這裡的狀態結果，持倉並非filled
                
                # 按照instId与ordId取消订单
                set_cancel_order_result = trade.set_cancel_order(
                    instId = 'INJ-USDT-SWAP', ordId = set_order_swap_limit_open_result['data'][0]['ordId']
                )
                print("沒有成功持多倉，撤單")
                
            # 若成功持倉，設置平倉價格為當下的價格上漲1%，以及止損價格1%
            elif order_result['data'][0]['state'] == "filled":
                # 不會重複開倉
                short_mark = False

                print(f"已經設定空單止盈止損，槓桿為 {LEVER}")
        
            # 緩衝
            time.sleep(50)
            
        else:
            
            print("沒有動作")
            # 由於最低獲取間隔為1m，總緩衝需60秒
            time.sleep(60)


當下價格: 39.618
震幅: -0.537%
USDT balance: 90.8913650713004
沒有持倉，可開倉


KeyboardInterrupt: 